In [1]:
import wfdb
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
def rollingWindow(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [3]:
MIN_BPM = 30
MIN_FREQ = 300  # from inspection of '.hea' content
SAMPLE_WINDOW = int(60 / MIN_BPM * MIN_FREQ)
FIRST_WINDOW = int(SAMPLE_WINDOW / 2)  # avoid artifacts in the beginning of signal

In [4]:
t17 = pd.read_csv('./data/training2017/REFERENCE.csv', names=['id', 'type'])
t17['fname'] = './data/training2017/' + t17.id
t17['noisy'] = t17.type == '~'

saa = pd.read_csv('./data/set-a/RECORDS-acceptable', names=['id'])
saa['fname'] = './data/set-a/' + saa.id.astype(str)
saa['noisy'] = False
sau = pd.read_csv('./data/set-a/RECORDS-unacceptable', names=['id'])
sau['fname'] = './data/set-a/' + sau.id.astype(str)
sau['noisy'] = True
sa = pd.concat([saa, sau])

val = pd.read_csv('./data/validation/REFERENCE.csv', names=['id', 'type'])
val['fname'] = './data/validation/' + val.id
val['noisy'] = val.type == '~'

In [5]:
train_validate_noisy = []
train_validate_clean = []

for idx, row in t17.iterrows():
    data, info = wfdb.rdsamp(row['fname'])
    for sgnl in data.T:
        sgnl_wnd = rollingWindow(sgnl[FIRST_WINDOW:], SAMPLE_WINDOW)
        sgnl_picks = np.random.randint(0, sgnl_wnd.shape[0], [int(sgnl_wnd.shape[0] / SAMPLE_WINDOW * 0.5)])
        if row['noisy']:
            train_validate_noisy.append(sgnl_wnd[sgnl_picks])
        else:
            train_validate_clean.append(sgnl_wnd[sgnl_picks])
for idx, row in sa.iterrows():
    data, info = wfdb.rdsamp(row['fname'])
    for sgnl in data.T:
        sgnl_ds = signal.resample(sgnl, int(len(sgnl) / info['fs'] * MIN_FREQ))
        sgnl_wnd = rollingWindow(sgnl_ds[FIRST_WINDOW:], SAMPLE_WINDOW)
        sgnl_picks = np.random.randint(0, sgnl_wnd.shape[0], [int(sgnl_wnd.shape[0] / SAMPLE_WINDOW * 0.5)])
        if row['noisy']:
            train_validate_noisy.append(sgnl_wnd[sgnl_picks])
        else:
            train_validate_clean.append(sgnl_wnd[sgnl_picks])
train_validate_noisy = np.concatenate(train_validate_noisy, 0)
train_validate_clean = np.concatenate(train_validate_clean, 0)

test_noisy = []
test_clean = []
for idx, row in val.iterrows():
    data, info = wfdb.rdsamp(row['fname'])
    for sgnl in data.T:
        sgnl_wnd = rollingWindow(sgnl[FIRST_WINDOW:], SAMPLE_WINDOW)
        sgnl_picks = np.random.randint(0, sgnl_wnd.shape[0], [int(sgnl_wnd.shape[0] / SAMPLE_WINDOW * 0.5)])
        if row['noisy']:
            test_noisy.append(sgnl_wnd[sgnl_picks])
        else:
            test_clean.append(sgnl_wnd[sgnl_picks])
test_noisy = np.concatenate(test_noisy, 0)
test_clean = np.concatenate(test_clean, 0)

In [6]:
latent_dim = 3
inner_dim = 32

def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = tf.random.normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + z_log_sigma * epsilon

# Encoder
input_x = tf.keras.layers.Input(shape= (SAMPLE_WINDOW, 1)) 
h = tf.keras.layers.LSTM(inner_dim, activation='relu')(input_x)

# Z
z_mean = tf.keras.layers.Dense(latent_dim)(h)
z_log_sigma = tf.keras.layers.Dense(latent_dim)(h)
z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_sigma])

# Decoder
decoder = tf.keras.layers.RepeatVector(SAMPLE_WINDOW)(z)
decoder = tf.keras.layers.LSTM(inner_dim, activation='relu', return_sequences=True)(decoder)
decoder = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder)

def vae_loss(input_x, decoder, z_log_sigma, z_mean):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    recon = tf.reduce_mean(tf.keras.metrics.binary_crossentropy(input_x, decoder))
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * tf.reduce_mean(tf.exp(z_log_sigma) + tf.square(z_mean) - 1. - z_log_sigma)

    return recon + kl

model = tf.keras.Model(input_x, decoder)
model.add_loss(vae_loss(input_x, decoder, z_log_sigma, z_mean))
model.compile(loss=None, optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001))

In [7]:
lenClean = train_validate_clean.shape[0]

trainX, valX = train_validate_clean[:int(0.85 * lenClean)], train_validate_clean[int(0.85 * lenClean):]

In [8]:
BATCH_SIZE = 512
SHUFFLE_SIZE = BATCH_SIZE * 4

train_dataset = tf.data.Dataset.from_tensor_slices(trainX[..., np.newaxis].astype(np.float32)).shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(valX[..., np.newaxis].astype(np.float32)).batch(BATCH_SIZE)

In [9]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10)

Epoch 1/10
109/109 [==============================] - 522s 5s/step - loss: 0.0953 - val_loss: inf
Epoch 2/10
109/109 [==============================] - 517s 5s/step - loss: 0.0820 - val_loss: inf
Epoch 3/10
109/109 [==============================] - 515s 5s/step - loss: 0.0738 - val_loss: inf
Epoch 4/10
109/109 [==============================] - 490s 4s/step - loss: 0.0675 - val_loss: inf
Epoch 5/10
109/109 [==============================] - 489s 4s/step - loss: 0.0652 - val_loss: inf
Epoch 6/10
109/109 [==============================] - 483s 4s/step - loss: 0.0625 - val_loss: inf
Epoch 7/10
109/109 [==============================] - 478s 4s/step - loss: 0.0608 - val_loss: inf
Epoch 8/10
109/109 [==============================] - 487s 4s/step - loss: 0.0599 - val_loss: inf
Epoch 9/10
109/109 [==============================] - 501s 5s/step - loss: 0.0586 - val_loss: inf
Epoch 10/10
109/109 [==============================] - 488s 4s/step - loss: 0.0566 - val_loss: inf


In [10]:
preds = model.predict(valX[..., np.newaxis].astype(np.float32))

307/307 [==============================] - 120s 391ms/step


In [26]:
(np.square(valX[..., np.newaxis] - preds).mean(1) > 0.1).mean()

0.07147962830593281

In [16]:
preds_noise = model.predict(train_validate_noisy[..., np.newaxis].astype(np.float32))

127/127 [==============================] - 50s 393ms/step


In [27]:
(np.square(train_validate_noisy[..., np.newaxis] - preds_noise).mean(1) > 0.1).mean()

0.4482417038137692

The above results demonstrate that the VAE:
1. classifies 7% of the subsignals in the clean signals as anomalous
2. classifies 44% of the subsignals in the noisy signals as anomalous

These results are comparable to the classifier in [ecgV1.ipynb](./ecgV1.ipynb), but I have not done much fine-tuning to this model yet.

Besides that, it is much slower than our classifier. However, it looks promising given that it has only been trained on the clean signals, which could also contain noisy segments.

A use of the VAE can be seen in [noisySegmentsVAE.ipynb](./noisySegmentsVAE.ipynb)

In [28]:
model.save_weights('./mydata/vae/0010.ckpt')